In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from skimage.filters import threshold_mean
import scipy.stats
import dabest

import sys
sys.path.append('../')
from src.functions import get_parameters

# parameters

In [ ]:
parameter_local_path = '../conf/local/parameters.yml'

parameters = get_parameters(parameter_local_path)

base_folder = parameters["data_dir"]
out_folder = parameters["out_dir"]



In [ ]:
px_per_cell = 300.0


condition_palette = {"ctrl" : "#1f77b4",
                       "Smad4 KO" : "#ff7f0e",
                        "Alk1 KO" : "#2ca02c"}

time_condition_palette={ 'ctrl P8' : "#1f77b4",  'ctrl P15': "#1f77b4", 
                         'Smad4 KO P8' : "#ff7f0e",   'Smad4 KO P15' : "#ff7f0e", 
                         'Alk1 KO P8' : "#2ca02c",   'Alk1 KO P15': "#2ca02c"}

time_series_palette1 = { 'ctrl P8 (P5)' : '#ccbb44',
                         'Smad4 KO P8 (P5)' : '#ccbb44', 
                         'Alk1 KO P8 (P5)' : '#ccbb44',
                         'ctrl P15 (P5)' : '#4477aa',
                         'Smad4 KO P15 (P5)' : '#4477aa', 
                         'Alk1 KO P15 (P5)' : '#4477aa'}
                         # 'Ctr P8 (P5)' : '#228833',
                         #'Cdc42 KO P8 (P5)' : '#228833', 
                         #'Rac1 KO P8 (P5)' : '#228833',
                         #'Ctr P9 (P5)' : '#ee6677',
                         #'Cdc42 KO P9 (P5)' : '#ee6677', 
                         #'Rac1 KO P9 (P5)' : '#ee6677',
                         #'Ctr P9 (P8)' : '#bbbbbb',
                         #'Cdc42 KO P9 (P8)' : '#bbbbbb', 
                         #'Rac1 KO P9 (P8)' : '#bbbbbb'}

time_series_palette2 = { 'P8 (inj. P5)' : '#ccbb44',  
                      'P15 (inj. P5)' : '#4477aa'} 
                    #  'P8 (inj. P5)' : '#228833',   
                    # 'P9 (inj. P5)' : '#ee6677',
                    # 'P9 (inj. P8)' : '#bbbbbb'}

swarm_ylim=(0, 15)
contrast_ylim=(-4, 2)

#effect_size = "cohens_d"
#effect_size = "hedges_g"
effect_size = "mean_diff"

# load data

In [ ]:
summary_data = pd.read_csv(out_folder + "summary_data.csv")
dtype_change = {'ExperimentID': int, 'collection_time': int}
summary_data = summary_data.astype(dtype_change)
summary_data

In [ ]:
summary_data["percentage_artery"] = 100.0*summary_data["GFP_artery_px"]/summary_data["GFP_px"]
summary_data["percentage_vein"] = 100.0*summary_data["GFP_vein_px"]/summary_data["GFP_px"]
summary_data["percentage_arterial_bed"] = 100.0*summary_data["GFP_arterial_bed_px"]/summary_data["GFP_px"]
summary_data["percentage_venous_bed"] = 100.0*summary_data["GFP_venous_bed_px"]/summary_data["GFP_px"]
#summary_data["percentage_GFP"] = 100.0*summary_data["GFP_px"]/summary_data["IB4_px"] #may not be very important


summary_data["labelled_ECs"] = summary_data["GFP_px"]/px_per_cell
summary_data["labelled_arterial_ECs"] = summary_data["GFP_artery_px"]/px_per_cell
summary_data["labelled_venous_ECs"] = summary_data["GFP_vein_px"]/px_per_cell

summary_data_P5_P8 = summary_data[summary_data["collection_time"]==8]
summary_data_P5_P8["inj_time"] = 'P8 (inj. P5)'
summary_data_P5_P15 = summary_data[summary_data["collection_time"]==15]
summary_data_P5_P15["inj_time"] = 'P15 (inj. P5)'
summary_data_ts = summary_data_P5_P8.copy()
summary_data_ts = summary_data_ts.append(summary_data_P5_P15, ignore_index = True)


summary_data_CTR = summary_data[summary_data["condition"]== "control"]
summary_data_CTR["cond."] = "ctrl"
summary_data_SMAD4 = summary_data[summary_data["condition"]== "SMAD4-KO"]
summary_data_SMAD4["cond."] = "Smad4 KO"
summary_data_Alk1 = summary_data[summary_data["condition"]== "ALK1-KO"]
summary_data_Alk1["cond."] = "Alk1 KO"

summary_data = summary_data_CTR.append(summary_data_SMAD4, ignore_index = True)
summary_data = summary_data.append(summary_data_Alk1, ignore_index = True)


# get mean values and SEM for reporting

In [ ]:
summary_data.groupby(by=["condition","collection_time"]).mean()
summary_data_mean=summary_data.groupby(by=["condition","collection_time"]).mean().add_suffix('').reset_index()
display(summary_data_mean)
summary_data_mean.to_csv(out_folder + 'summary_data_mean.csv')

#summary_data.groupby(by=["time_interval"]).mean()

In [ ]:
summary_data.groupby(by=["condition","collection_time"]).sem()
summary_data_sem=summary_data.groupby(by=["condition","collection_time"]).sem().add_suffix('').reset_index()
display(summary_data_sem)
summary_data_sem.to_csv(out_folder + 'summary_data_sem.csv')

#summary_data.groupby(by=["time_interval"]).sem()

In [ ]:
control_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='control']['percentage_artery'])
control_p15 = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='control']['percentage_artery'])
SMAD4_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='SMAD4-KO']['percentage_artery'])
SMAD4_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='SMAD4-KO']['percentage_artery'])
Alk1_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='ALK1-KO']['percentage_artery'])
Alk1_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='ALK1-KO']['percentage_artery'])


plot_df = pd.DataFrame(data = {'percentage_artery':control_p8})
plot_df['condition'] = 'ctrl P8'
temp_df = pd.DataFrame(data = {'percentage_artery':control_p15})
temp_df['condition'] = 'ctrl P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_artery':SMAD4_p8})
temp_df['condition'] = 'Smad4 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_artery':SMAD4_p15})
temp_df['condition'] = 'Smad4 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_artery':Alk1_p8})
temp_df['condition'] = 'Alk1 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_artery':Alk1_p15})
temp_df['condition'] = 'Alk1 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)

print(plot_df["condition"].unique())
#plot_df

properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_artery",
                          idx=(( 'ctrl P8', 'ctrl P15'), 
                               ( 'Smad4 KO P8',  'Smad4 KO P15'), 
                               ( 'Alk1 KO P8',  'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test.savefig(out_folder + "percentage_artery_P8_P15.pdf")
test.savefig(out_folder + "percentage_artery_P8_P15.png")

In [ ]:
properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_artery",
                          idx=(( 'ctrl P15',  'Smad4 KO P15',  'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test.savefig(out_folder + "percentage_artery_P15.pdf")
test.savefig(out_folder + "percentage_artery_P15.png")

In [ ]:
properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_artery",
                          idx=(( 'ctrl P8',  'Smad4 KO P8',  'Alk1 KO P8')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test.savefig(out_folder + "percentage_artery_P8.pdf")
test.savefig(out_folder + "percentage_artery_P8.png")

In [ ]:
control_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='control']['percentage_arterial_bed'])
control_p15 = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='control']['percentage_arterial_bed'])
SMAD4_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='SMAD4-KO']['percentage_arterial_bed'])
SMAD4_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='SMAD4-KO']['percentage_arterial_bed'])
Alk1_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='ALK1-KO']['percentage_arterial_bed'])
Alk1_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='ALK1-KO']['percentage_arterial_bed'])


plot_df = pd.DataFrame(data = {'percentage_arterial_bed':control_p8})
plot_df['condition'] = 'ctrl P8'
temp_df = pd.DataFrame(data = {'percentage_arterial_bed':control_p15})
temp_df['condition'] = 'ctrl P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_arterial_bed':SMAD4_p8})
temp_df['condition'] = 'Smad4 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_arterial_bed':SMAD4_p15})
temp_df['condition'] = 'Smad4 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_arterial_bed':Alk1_p8})
temp_df['condition'] = 'Alk1 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_arterial_bed':Alk1_p15})
temp_df['condition'] = 'Alk1 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)

print(plot_df["condition"].unique())
#plot_df

properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_arterial_bed",
                          idx=(( 'ctrl P8', 'ctrl P15'), 
                               ( 'Smad4 KO P8',  'Smad4 KO P15'), 
                               ( 'Alk1 KO P8',  'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
#test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test = properties_dabest.mean_diff.plot(custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test.savefig(out_folder + "percentage_arterial_bed_P8_P15.pdf")
test.savefig(out_folder + "percentage_arterial_bed_P8_P15.png")

In [ ]:

properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_arterial_bed",
                          idx=(( 'ctrl P15',  'Smad4 KO P15',  'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
#test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test = properties_dabest.mean_diff.plot(custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));

test.savefig(out_folder + "percentage_arterial_bed_P15.pdf")
test.savefig(out_folder + "percentage_arterial_bed_P15.png")

In [ ]:
properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_arterial_bed",
                          idx=(( 'ctrl P8',  'Smad4 KO P8',  'Alk1 KO P8')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
test = properties_dabest.mean_diff.plot(custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test.savefig(out_folder + "percentage_arterial_bed_P8.pdf")
test.savefig(out_folder + "percentage_arterial_bed_P8.png")

In [ ]:
control_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='control']['percentage_vein'])
control_p15 = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='control']['percentage_vein'])
SMAD4_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='SMAD4-KO']['percentage_vein'])
SMAD4_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='SMAD4-KO']['percentage_vein'])
Alk1_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='ALK1-KO']['percentage_vein'])
Alk1_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='ALK1-KO']['percentage_vein'])


plot_df = pd.DataFrame(data = {'percentage_vein':control_p8})
plot_df['condition'] = 'ctrl P8'
temp_df = pd.DataFrame(data = {'percentage_vein':control_p15})
temp_df['condition'] = 'ctrl P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_vein':SMAD4_p8})
temp_df['condition'] = 'Smad4 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_vein':SMAD4_p15})
temp_df['condition'] = 'Smad4 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_vein':Alk1_p8})
temp_df['condition'] = 'Alk1 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_vein':Alk1_p15})
temp_df['condition'] = 'Alk1 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)

properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_vein",
                          idx=(( 'ctrl P8',  'ctrl P15'), 
                               ( 'Smad4 KO P8',   'Smad4 KO P15'), 
                               ( 'Alk1 KO P8',   'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));

test.savefig(out_folder + "percentage_vein_P8_P15.pdf")
test.savefig(out_folder + "percentage_vein_P8_P15.png")

In [ ]:
properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_vein",
                          idx=(( 'ctrl P15',  'Smad4 KO P15',  'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test.savefig(out_folder + "percentage_vein_P15.pdf")
test.savefig(out_folder + "percentage_vein_P15.png")

In [ ]:
control_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='control']['percentage_venous_bed'])
control_p15 = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='control']['percentage_venous_bed'])
SMAD4_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='SMAD4-KO']['percentage_venous_bed'])
SMAD4_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='SMAD4-KO']['percentage_venous_bed'])
Alk1_p8 = np.array(summary_data_P5_P8[summary_data_P5_P8['condition']=='ALK1-KO']['percentage_venous_bed'])
Alk1_p15  = np.array(summary_data_P5_P15[summary_data_P5_P15['condition']=='ALK1-KO']['percentage_venous_bed'])


plot_df = pd.DataFrame(data = {'percentage_venous_bed':control_p8})
plot_df['condition'] = 'ctrl P8'
temp_df = pd.DataFrame(data = {'percentage_venous_bed':control_p15})
temp_df['condition'] = 'ctrl P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_venous_bed':SMAD4_p8})
temp_df['condition'] = 'Smad4 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_venous_bed':SMAD4_p15})
temp_df['condition'] = 'Smad4 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_venous_bed':Alk1_p8})
temp_df['condition'] = 'Alk1 KO P8'
plot_df = plot_df.append(temp_df, ignore_index = True)
temp_df = pd.DataFrame(data = {'percentage_venous_bed':Alk1_p15})
temp_df['condition'] = 'Alk1 KO P15'
plot_df = plot_df.append(temp_df, ignore_index = True)

properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_venous_bed",
                          idx=(( 'ctrl P8',  'ctrl P15'), 
                               ( 'Smad4 KO P8',   'Smad4 KO P15'), 
                               ( 'Alk1 KO P8',   'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test = properties_dabest.mean_diff.plot(custom_palette = time_condition_palette) 
test.savefig(out_folder + "percentage_venous_bed_P8_P15.pdf")
test.savefig(out_folder + "percentage_venous_bed_P8_P15.png")

In [ ]:
properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_venous_bed",
                          idx=(( 'ctrl P15',  'Smad4 KO P15',  'Alk1 KO P15')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
#test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test = properties_dabest.mean_diff.plot( custom_palette = time_condition_palette)
test.savefig(out_folder + "percentage_venous_bed_P15.pdf")
test.savefig(out_folder + "percentage_venous_bed_P15.png")

In [ ]:
properties_dabest = dabest.load(data = plot_df, x="condition", y="percentage_venous_bed",
                          idx=(( 'ctrl P8',  'Smad4 KO P8',  'Alk1 KO P8')))

print(properties_dabest.mean_diff)
display(properties_dabest.mean_diff.results)

#properties_dabest = dabest.load(data = summary_data_CTR, x="time_interval", y="labelled_arterial_ECs",
#                          idx=('P8_P9', 'P5_P6', 'P5_P7', 'P5_P8', 'P5_P9'))

# Produce a Cumming estimation plot.
#test = properties_dabest.mean_diff.plot(swarm_ylim=(0, 10), contrast_ylim=(-3, 2));
#test = properties_dabest.mean_diff.plot(swarm_ylim=swarm_ylim, custom_palette = time_condition_palette) #, contrast_ylim=(-4, 2));
test = properties_dabest.mean_diff.plot( custom_palette = time_condition_palette)
test.savefig(out_folder + "percentage_venous_bed_P8.pdf")
test.savefig(out_folder + "percentage_venous_bed_P8.png")